# component_from_yaml


We can define a place and route component by a netlist in YAML format

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
"""

c = pp.component_from_yaml(yaml)
c

In [ ]:
c.instances

In [ ]:
c.instances['mmi_long'].x = 100

In [ ]:
c.show()
c.plot()

You can also do that by using `get_component_from_yaml` from the Pdk class.

In [ ]:
import pp
from pp.pdk import PDK_NITRIDE_C

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
"""

pdk = PDK_NITRIDE_C
c = pdk.get_component_from_yaml(yaml)
c

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: W0
        x: 20
        y: 10
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: W0
        x: 20
        y: 10
        mirror: True
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: W0
        x: 20
        y: 10
        mirror: True
        
ports:
    W0: mmi_long,E0
    W1: mmi_long,E1
    E0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
        mirror: W0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
        mirror: W0
ports:
    W0: mmi_long,E0
    W1: mmi_long,E1
    E0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
        mirror: 25
ports:
    W0: mmi_long,E0
    W1: mmi_long,E1
    E0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: W0
        x: 10
        y: 20
        rotation: 90
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: W0
        x: 10
        y: 20
        rotation: 90
ports:
    N0: mmi_long,E0
    N1: mmi_long,E1
    S0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
        rotation: 90
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
c.size_info.north

In [ ]:
c.size_info.east

In [ ]:
import pp

yaml = """
instances:
    b:
      component: bend_circular
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:
    mmi_short:
        port: W0
        x: 10
        y: 20
connections:
    b,W0 : mmi_short,E1
    mmi_long,W0: b, N0

ports:
    W0: mmi_short,W0
"""


c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
from omegaconf import OmegaConf
import io

In [ ]:
d = OmegaConf.load(io.StringIO(
"""
x: mmi_short,E0 10
dx: 10

"""
))
d

In [ ]:
d.keys()

## Relative port placing with respect to another instance port

You can also define an x and y offset with `dx` and `dy`

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5

placements:
    mmi_short:
        port: W0
        x: 0
        y: 0
    mmi_long:
        port: W0
        x: mmi_short,E1
        y: mmi_short,E1
        dx : 10
        dy: -10
"""


c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5

placements:
    mmi_short:
        port: W0
        x: 0
        y: 0
    mmi_long:
        port: W0
        x: mmi_short,E1
        y: mmi_short,E1
        dx: 10
        dy: -10
"""


c = pp.component_from_yaml(yaml)
c.show()
c.plot()

### Cicular reference warning

You have to be careful not doing a circuilar reference

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5

placements:
    mmi_short:
        port: W0
        x: mmi_long,E1
        y: mmi_long,E1
    mmi_long:
        port: W0
        x: mmi_short,E1
        y: mmi_short,E1
        dx : 10
        dy: 20
"""

pp.component_from_yaml(yaml) 

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:
    mmi_long:
        x: 100
        y: 100
routes:
    optical:
        factory: optical
        links:
            mmi_short,E1: mmi_long,W0
"""


c = pp.component_from_yaml(yaml)
c.show()
c.plot()

You can **rotate** and instance specifying the angle in degrees

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 150
        y: 100
routes:
    optical:
        factory: optical
        links:
            mmi_short,E1: mmi_long,E0
"""


c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
from pp.pdk import PDK_NITRIDE_C

pdk = PDK_NITRIDE_C

c = pdk.get_component_from_yaml(yaml)
c

You can also define ports for the component

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
        
routes:
    optical:
        factory: optical
        links:
            mmi_short,E1: mmi_long,E0
    
ports:
    E0: mmi_short,W0
    W0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
r = c.routes['mmi_short,E1:mmi_long,E0']
r

In [ ]:
c.instances

## YAML instances, placements, connections, ports, routes


We can define the netlist connections of a component by a netlist in YAML format

Note that you define the connections as `instance_source.port ->
instance_destination.port` so the order is important and therefore you can only
change the position of the `instance_destination`

For example, this coupler has the center coupling region at (100, 0)

In [ ]:
pp.components.coupler_symmetric()

In [ ]:
import pp

gap = 0.2
wg_width = 0.5
length = 10

yaml = f"""
instances:
    left:
      component: coupler_symmetric
      settings:
        gap: {gap}
        width: {wg_width}
    right:
      component: coupler_symmetric
      settings:
        gap: {gap}
        width: {wg_width}
    center:
      component: coupler_straight
      settings:
        gap: {gap}
        width: {wg_width}
        length: {length}

placements:
    center:
        x: 100
        y: 0
    left:
        mirror: True

connections:
    center,W0: left,W0
    right,W0: center,E0

ports:
    w0: left,E0
    w1: left,E1
    e0: right,E0
    e1: right,E1

"""

"""
    left,W0: center,W0
    right,W0: center,E0

"""

c = pp.component_from_yaml(yaml)
c

While this one has the sbend_left_coupler `sl` centered at (100, 0)

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 100
        y: 100
"""

c = pp.component_from_yaml(yaml)
c

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:
    mmi_long:
        x: 100
        y: 100
routes:
    route1:
        factory: optical
        links:
            mmi_short,E1: mmi_long,W0
"""


c = pp.component_from_yaml(yaml)
c

You can **rotate** and instance specifying the angle in degrees

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
routes:
    mmi_connect:
        factory: optical
        links:
            mmi_short,E1: mmi_long,E0
"""


c = pp.component_from_yaml(yaml)
c

You can also define ports for the component

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
        
routes:
    optical:
        factory: optical
        links:
            mmi_short,E1: mmi_long,E0
    
ports:
    E0: mmi_short,W0
    W0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
c

In [ ]:
c.routes

In [ ]:
r = c.routes['mmi_short,E1:mmi_long,E0']
r

In [ ]:
c.instances

In [ ]:
c.routes

## get_bundle routes

You can define several `bundle_routes` 

In [ ]:
import pp

sample_2x2_connections_solution = """
name:
    connections_2x2_problem

instances:
    mmi_bottom:
      component: mmi2x2
    mmi_top:
      component: mmi2x2

placements:
    mmi_top:
        x: 100
        y: 100

routes:
    optical:
        factory: optical
        links:
            mmi_bottom,E0: mmi_top,W0
            mmi_bottom,E1: mmi_top,W1

"""

c = pp.component_from_yaml(sample_2x2_connections_solution)
c

## Custom component factories

In [ ]:
@pp.cell
def pad_new(size=(100, 100), layer=pp.LAYER.M3):
    c = pp.Component()
    compass = c << pp.components.compass(size=size, layer=layer)
    c.ports = compass.ports
    return c

c = pad_new(cache=False)
c

In [ ]:
import pp
from pp.components import component_factory

component_factory.update(pad_new=pad_new)
print('pad_new' in component_factory)

sample_custom_component = """
name:
    connections_2x2_problem

instances:
    bot:
      component: pad_new
    top:
      component: pad_new

placements:
    top:
        x: 0
        y: 200
"""

c = pp.component_from_yaml(sample_custom_component, component_factory=component_factory)
c

## custom route factories

In [ ]:
import pp
from pp.components import component_factory
from pp.routing.factories import route_factory
from pp.routing.manhattan import round_corners

component_factory.update(pad_new=pad_new)
print('pad_new' in component_factory)


def connect_electrical_new(
    way_points= [],
    bend_factory = pp.components.corner,
    straight_factory= pp.components.wire,
    wg_width=10,
    bend_radius = 0.1,
    **kwargs,
):
    """ Returns a custom electrical route
    """
    bend90 = pp.call_if_func(bend_factory, radius=bend_radius, width=wg_width, layer=pp.LAYER.M3)
    connector = round_corners(way_points, bend_factory=bend90, straight_factory=straight_factory)
    return connector


route_factory.update(electrical_new=connect_electrical_new)


sample_custom_component = """
name:
    custom_routes

instances:
    bl:
      component: pad_new
    tl:
      component: pad_new
    br:
      component: pad_new
    tr:
      component: pad_new

placements:
    tl:
        x: 0
        y: 200

    br:
        x: 400
        y: 400

    tr:
        x: 400
        y: 600

routes:
    electrical1:
        factory: electrical_new
        links:
            bl,E: br,W
            tl,E: tr,W
    electrical2:
        factory: electrical_new
        links:
            bl,S: br,E
    optical:
        factory: optical
        links:
            tl,N: tr,N

"""

pp.component_from_yaml(sample_custom_component, component_factory=component_factory, route_factory=route_factory)

In [ ]:
sample_custom_component = """
name:
    custom_routes

instances:
    t:
      component: pad_array
      settings:
          port_list: ['S']
    b:
      component: pad_array

placements:
    t:
        x: 200
        y: 400
routes:
    electrical:
        factory: electrical
        links:
            t,S0: b,N0
            t,S5: b,N5
"""


c = pp.component_from_yaml(sample_custom_component)
c

## Route settings

Also, you can define route aliases, that have different settings and specify the route `factory` as a parameter as well as the `settings` for that particular route alias.

In [ ]:
import pp

sample_settings = """
name: sample_settings

instances:
    bl:
      component: pad
    tl:
      component: pad
    br:
      component: pad
    tr:
      component: pad

placements:
    tl:
        x: 0
        y: 200

    br:
        x: 400
        y: 400

    tr:
        x: 400
        y: 600

routes:
    optical_r100:
        factory: optical
        settings:
            bend_radius: 100
        links:
            tl,N: tr,N
    optical_r200:
        factory: optical
        settings:
            bend_radius: 200
        links:
            bl,S: br,E
"""
pp.component_from_yaml(sample_settings)

In [ ]:
sample_custom_component = """

instances:
    t:
      component: pad_array
      settings:
          port_list: ['S']
    b:
      component: pad_array

placements:
    t:
        x: 200
        y: 400
routes:
    optical:
        factory: optical
        settings: 
            bend_radius: 50
        links:
            t,S0: b,N0
            t,S5: b,N5
"""

pp.component_from_yaml(sample_custom_component)

In [ ]:
import pp

sample = """

instances:
    t:
      component: pad_array
      settings:
          port_list: ['S']
    b:
      component: pad_array

placements:
    t:
        x: 100
        y: 1000
routes:
    route1:
        factory: optical
        link_factory: link_ports_path_length_match
        link_settings:
            extra_length: 200
        links:
            t,S0: b,N0
            t,S5: b,N4
"""

c = pp.component_from_yaml(sample)
print(c.routes['t,S0:b,N0'])
c

In [ ]:
import pp

sample = """

instances:
    t:
      component: pad_array
      settings:
          port_list: ['S']
    b:
      component: pad_array

placements:
    t:
        x: -250
        y: 1000
routes:
    route1:
        factory: optical
        link_factory: link_optical_waypoints
        link_settings:
            waypoints: 
            - [0,0]
            - [0, 600]
            - [-250, 600]
            - [-250, 1000]
        links:
            t,S5: b,N4
"""

c = pp.component_from_yaml(sample)
c

In [ ]:
import pp

sample = """

instances:
    t:
      component: pad_array
      settings:
          port_list: ['S']
    b:
      component: pad_array

placements:
    t:
        x: -250
        y: 1000
routes:
    route1:
        factory: optical
        link_factory: link_optical_waypoints
        link_settings:
            waypoints:  [[0,0], [0, 600], [-250, 600], [-250, 1000]]
        links:
            t,S5: b,N4
"""

c = pp.component_from_yaml(sample)
c

In [ ]:
from omegaconf import OmegaConf
import io

d = OmegaConf.load(io.StringIO("""
way_points: 
    - [0,0]
    - [0, 600]
    - [-250, 600]
    - [-250, 1000]

demo: a
"""))

In [ ]:
d = OmegaConf.load(io.StringIO("way_points:  [[0,0], [0, 600], [-250, 600], [-250, 1000]]"))

In [ ]:
d

In [ ]:
import numpy as np

c = pp.Component('waypoints_sample')
route = pp.routing.get_route_from_waypoints(
    waypoints=np.array(d['way_points'])
    #route_filter=pp.routing.connect_strip,
)

c.add(route['references'])
c

## YAML netlist

Note that you define the connections as `instance_source.port -> instance_destination.port` so the order is important and therefore you can only change the position of the `instance_destination`

In [ ]:
import pp

netlist = """
instances:
    CP1:
      component: mmi1x2
      settings:
          width_mmi: 4.5
          length_mmi: 10
    CP2:
        component: mmi1x2
        settings:
            width_mmi: 4.5
            length_mmi: 5
    arm_top:
        component: mzi_arm
    arm_bot:
        component: mzi_arm

placements:
    arm_bot:
        mirror: True
    CP2:
        mirror: True
ports:
    W0: CP1,W0
    E0: CP2,W0

connections:
    arm_bot,W0: CP1,E0 
    arm_top,W0: CP1,E1
    CP2,E0: arm_bot,E0 
    CP2,E1: arm_top,E0
"""

c = pp.component_from_yaml(netlist)
c.show()
c.plot()

We can reduce the length of each of the arms

In [ ]:
import pp

netlist = """
instances:
    CP1:
      component: mmi1x2
      settings:
          width_mmi: 4.5
          length_mmi: 10
    CP2:
        component: mmi1x2
        settings:
            width_mmi: 4.5
            length_mmi: 5
    arm_top:
        component: mzi_arm
        settings:
            L0: 0
            DL: 0
    arm_bot:
        component: mzi_arm
        settings:
            L0: 0
            DL: 10

placements:
    arm_bot:
        mirror: True
    CP2:
        mirror: True
ports:
    W0: CP1,W0
    E0: CP2,W0

connections:
    arm_bot,W0: CP1,E0 
    arm_top,W0: CP1,E1
    CP2,E0: arm_bot,E0 
    CP2,E1: arm_top,E0
"""

c = pp.component_from_yaml(netlist)
c.show()
c.plot()

We can also use 2x2 couplers instead of 1x2 MMIs

In [ ]:
import pp

netlist = """
instances:
    CP1:
      component: mmi2x2
      settings:
          width_mmi: 4.5
          length_mmi: 10
    CP2:
        component: mmi2x2
        settings:
            width_mmi: 4.5
            length_mmi: 5
    arm_top:
        component: mzi_arm
        settings:
            L0: 0
            DL: 0
    arm_bot:
        component: mzi_arm
        settings:
            L0: 0
            DL: 10

placements:
    arm_bot:
        mirror: True
    CP2:
        mirror: True
ports:
    W0: CP1,W0
    E0: CP2,W0
    W1: CP1,W1
    E1: CP2,W1

connections:
    arm_bot,W0: CP1,E0 
    arm_top,W0: CP1,E1
    CP2,E0: arm_bot,E0 
    CP2,E1: arm_top,E0
"""

c = pp.component_from_yaml(netlist)
c.show()
c.plot()

We can also expose more ports, such as the electrical ports, so we can route electrical signals to the circuits.

In [ ]:
import pp

netlist = """
instances:
    CP1:
      component: mmi2x2
      settings:
          width_mmi: 4.5
          length_mmi: 10
    CP2:
        component: mmi2x2
        settings:
            width_mmi: 4.5
            length_mmi: 5
    arm_top:
        component: mzi_arm
        settings:
            L0: 0
            DL: 0
    arm_bot:
        component: mzi_arm
        settings:
            L0: 0
            DL: 10

placements:
    arm_bot:
        mirror: True
    CP2:
        mirror: True
ports:
    W0: CP1,W0
    E0: CP2,W0
    W1: CP1,W1
    E1: CP2,W1
    E_TOP_0: arm_top,E_0
    E_TOP_1: arm_top,E_1
    E_TOP_2: arm_top,E_2
    E_TOP_3: arm_top,E_3
    E_BOT_0: arm_bot,E_0
    E_BOT_1: arm_bot,E_1
    E_BOT_2: arm_bot,E_2
    E_BOT_3: arm_bot,E_3

connections:
    arm_bot,W0: CP1,E0 
    arm_top,W0: CP1,E1
    CP2,E0: arm_bot,E0 
    CP2,E1: arm_top,E0
"""

c = pp.component_from_yaml(netlist)
c.show()
c.plot()

In [ ]:
c.ports

## Custom factories

You can leverage netlist defined components to define more complex circuits

In [ ]:
import pp

@pp.cell
def mzi_custom(delta_length=0):
    netlist = f"""
instances:
    CP1:
      component: mmi2x2
      settings:
          width_mmi: 4.5
          length_mmi: 10
    CP2:
        component: mmi2x2
        settings:
            width_mmi: 4.5
            length_mmi: 5
    arm_top:
        component: mzi_arm
        settings:
            L0: 0
            DL: 0
            with_elec_connections: False
    arm_bot:
        component: mzi_arm
        settings:
            L0: 0
            DL: {delta_length/2}
            with_elec_connections: False

placements:
    arm_bot:
        mirror: True
    CP2:
        mirror: True
ports:
    W0: CP1,W0
    E0: CP2,W0
    W1: CP1,W1
    E1: CP2,W1

connections:
    arm_bot,W0: CP1,E0 
    arm_top,W0: CP1,E1
    CP2,E0: arm_bot,E0 
    CP2,E1: arm_top,E0
"""
    return pp.component_from_yaml(netlist)


c = mzi_custom(delta_length=10, cache=False)
c.show()
c.plot()

In [ ]:
c.ports

In [ ]:
import pp

@pp.cell
def mzi_custom(delta_length):
    return pp.components.mzi(delta_length=delta_length/2, splitter=pp.components.mmi2x2)

pp.components.component_factory.update(dict(mzi_custom=mzi_custom))
c = pp.components.component_factory['mzi_custom'](delta_length=0, cache=False)
c.plot()
c.show()
print(c.ports.keys())

In [ ]:
import pp

@pp.cell
def mzi_filter(delta_lengths=(20, 40, 60), component_factory=pp.components.component_factory):
    sample = f"""
instances:
    mzi1:
      component: mzi_custom
      settings:
          delta_length: {delta_lengths[0]}

    arm_top1:
        component: mzi_arm
        settings:
            L0: 0
            DL: 0
            with_elec_connections: False
    arm_bot1:
        component: mzi_arm
        settings:
            L0: 0
            DL: {delta_lengths[1]/2}
            with_elec_connections: False

    mzi3:
        component: mzi_custom
        settings:
            delta_length: {delta_lengths[2]}

placements:
    arm_bot1:
        mirror: True

ports:
    W0: mzi1,W0
    E0: mzi3,E0
    W1: mzi1,W1
    E1: mzi3,E1

connections:
    arm_bot1,W0: mzi1,E0
    arm_top1,W0: mzi1,E1
    mzi3,W0: arm_bot1,E0
    mzi3,W1: arm_top1,E0
    
"""

    c = pp.component_from_yaml(sample, component_factory=component_factory)
    return c

c = mzi_filter(cache=False)
c.show()
c.plot()

In [ ]:
c = pp.components.mzi()

In [ ]:
c.plot_netlist()

In [ ]:
n = c.get_netlist()

In [ ]:
print(c.get_netlist_yaml())

## Netlist export

Any component exports its netlist `get_netlist` and returns an `OmegaConf` dict that can be easily converted into JSON and YAML

In [ ]:
import io
from omegaconf import OmegaConf
import pp

In [ ]:
c = pp.components.mzi()
c

In [ ]:
c.plot_netlist()

In [ ]:
netlist = c.get_netlist()

In [ ]:
c.write_netlist('mzi.yml', full_settings=True)

In [ ]:
n = OmegaConf.load('mzi.yml')

In [ ]:
i = list(n['instances'].keys())
i

In [ ]:
n['instances'][i[0]]['settings']

In [ ]:
import pp
from omegaconf import OmegaConf
import pathlib

c1 = pp.component_from_yaml('mzi.yml')
c1

In [ ]:
n = c1.get_netlist(full_settings=True)
connections = n['connections']
len(connections)

## Plot netlist

You can plot the netlist of components.

Every gdsfactory component can either be defined by its netlist or using layout friendly functions such as component sequence to define it and then `get_netlist()` method.

Connections are determined by extracting all the ports of a component, and asuming that ports with the same (x, y) are connected.

 When you do `get_netlist()` for a component it will only show connections for the instances that belong to that component (it trims the netlist). So despite having  a lot of connections, it will show only the meaningful connections for that component. For example, a ring has a ring_coupler. but if you want to digg deeper, the connections that made that ring coupler are still available.

In [ ]:
import pp

In [ ]:
c = pp.components.mzi()
c

In [ ]:
c = pp.components.mzi()
n = c.get_netlist()
print(c.get_netlist_yaml())

In [ ]:
c.plot_netlist()

In [ ]:
n.keys()

In [ ]:
import pp

yaml =  """
instances:

    mmi1x2_12_0:
        component: mmi1x2

    bend_circular_R10p00_32_4:
      component: bend_circular
    
    waveguide_L1p00_35_11:
        component: waveguide
        settings:
            length: 10
            layer: [2, 0]

connections:
    bend_circular_R10p00_32_4,W0: mmi1x2_12_0,E1
    waveguide_L1p00_35_11,W0: bend_circular_R10p00_32_4,N0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()
c.name = "mmi_with_bend_circular"
print(c.name)

In [ ]:
n = c.get_netlist()

In [ ]:
print(c.get_netlist_yaml())

In [ ]:
n['connections']

In [ ]:
c.plot_netlist()

In [ ]:
c = pp.components.mzi()
c.plot()
c.plot_netlist()

In [ ]:
c = pp.components.ring_single()
c

In [ ]:
c.plot_netlist()

In [ ]:
c = pp.components.ring_double()
c

In [ ]:
c.plot_netlist()

In [ ]:
import pp

c = pp.components.ring_single()
c

In [ ]:
c.plot_netlist()

In [ ]:
c = pp.components.ring_double()
c

In [ ]:
c.plot_netlist()

In [ ]:
print(c.get_netlist_yaml())

In [ ]:
c = pp.components.mzi()
c

In [ ]:
c.plot_netlist()

In [ ]:
c = pp.components.mzit()
c

In [ ]:
c.plot_netlist()

In [ ]:
c = pp.components.mzi_lattice()
c

In [ ]:
import pp
coupler_lengths = [10, 20, 30]
coupler_gaps = [0.1, 0.2, 0.3]
delta_lengths = [10, 100]

c = pp.components.mzi_lattice(coupler_lengths=coupler_lengths, coupler_gaps=coupler_gaps, delta_lengths=delta_lengths)
c

In [ ]:
print(c.get_netlist_yaml())

In [ ]:
c.plot_netlist()

In [ ]:
coupler_lengths = [10, 20, 30, 40]
coupler_gaps = [0.1, 0.2, 0.4, 0.5]
delta_lengths = [10, 100, 200]

c = pp.components.mzi_lattice(coupler_lengths=coupler_lengths, coupler_gaps=coupler_gaps, delta_lengths=delta_lengths)
c

In [ ]:
n = c.get_netlist()

In [ ]:
c.plot_netlist()